In [1]:
import io
import pandas as pd
import os
import sys
from sys import path
from tqdm import tqdm

In [10]:
df1 = pd.read_csv("Data/hist_info_DID_PAD_2016.csv")
df2 = pd.read_csv("Data/hist_info_PAD_DID_2016.csv")
df = pd.concat([df1, df2]).reset_index(drop=True)
df.head(-5)

,1.date,2.origin,3.destination,4.stops,RID,5.schedule_detail
0,2016-01-04,HFD,PAD,17,201601041343260,",location,gbtt_ptd,gbtt_pta,actual_td,actual_t..."
1,2016-01-04,DID,PAD,9,201601041341163,",location,gbtt_ptd,gbtt_pta,actual_td,actual_t..."
2,2016-01-04,BAN,PAD,18,201601041343383,",location,gbtt_ptd,gbtt_pta,actual_td,actual_t..."
3,2016-01-05,HFD,PAD,17,201601051375276,",location,gbtt_ptd,gbtt_pta,actual_td,actual_t..."
4,2016-01-05,BAN,PAD,18,201601051375397,",location,gbtt_ptd,gbtt_pta,actual_td,actual_t..."
...,...,...,...,...,...,...
49130,2016-12-23,PAD,BRI,7,201612236723370,",location,gbtt_ptd,gbtt_pta,actual_td,actual_t..."
49131,2016-12-30,PAD,BRI,7,201612306723370,",location,gbtt_ptd,gbtt_pta,actual_td,actual_t..."
49132,2016-12-30,PAD,SWA,11,201612306723367,",location,gbtt_ptd,gbtt_pta,actual_td,actual_t..."
49133,2016-12-30,PAD,OXF,7,201612306723389,",location,gbtt_ptd,gbtt_pta,actual_td,actual_t..."


In [3]:
# Assuming the notebook is in the same directory as your DataPreprocessing folder
current_path = os.getcwd()

dir_path = os.path.join(current_path, "DataPreprocessing")
sys.path.insert(0, dir_path)

from data_clean_02 import data_cleaning
from data_structure_next_n_stations_03 import data_structure_next_n_stations
from delay_mechanism_04 import delay_mechanism

In [11]:
df_schedule_detail_list = []
for i in tqdm(range(len(df))):
    df_schedule_detail = pd.read_csv(io.StringIO(df.iloc[i]['5.schedule_detail']), sep=',', dtype=str)
    df_schedule_detail = df_schedule_detail.drop(df_schedule_detail.columns[0], axis = 1)
    df_schedule_detail_list.append(df_schedule_detail)
# Drop that column
df.drop("5.schedule_detail", axis = 1, inplace = True)
# Put whatever series you want in its place
df["5.schedule_detail"] = df_schedule_detail_list

  0%|          | 0/49140 [00:00<?, ?it/s]

100%|██████████| 49140/49140 [02:09<00:00, 379.92it/s]


In [12]:
historical_information, station_dwell_time_unique, OD_pairs_unique = data_cleaning(df)
data_next_1_station = data_structure_next_n_stations(historical_information, station_dwell_time_unique, OD_pairs_unique, 1)
data_next_1_station = delay_mechanism(data_next_1_station)

2/18 Giving arrival/departure time for starting/terminating stations: 100%|██████████| 49140/49140 [00:57<00:00, 857.74it/s]
3/18 Dropping all rows with greater than 10 percent missing data: 100%|██████████| 49140/49140 [02:57<00:00, 276.23it/s]
4/18 Extract all trips to find unique trips: 100%|██████████| 46983/46983 [00:25<00:00, 1814.59it/s]
5/18 Calculate time taken to travel and dwell time of all trips: 100%|██████████| 46983/46983 [05:10<00:00, 151.48it/s]
9/18 Find average time for each OD pair:  21%|██        | 62/298 [00:06<00:25,  9.19it/s]d:\Wallace\wallace-azure-ml\DataPreprocessing\data_clean_02.py:231: RuntimeWarning: invalid value encountered in double_scalars
  average_time_travelled = total_time_travelled/len(df)
d:\Wallace\wallace-azure-ml\DataPreprocessing\data_clean_02.py:232: RuntimeWarning: invalid value encountered in double_scalars
  average_time_travelled_predicted = total_time_travelled_predicted/len(df)
11/18 Calculating the total number of nan values: 100%|█

Total null values = 0.0


12/18 Extracting missing information: 100%|██████████| 44805/44805 [00:05<00:00, 7616.22it/s]
13/18 Inputting actual and predicted travel time and dwell time: 100%|██████████| 4884/4884 [00:25<00:00, 192.37it/s]
14/18 filling in missing data for actual_arrival_time: 100%|██████████| 4884/4884 [00:09<00:00, 513.54it/s]
15/18 Extracting missing departure information: 100%|██████████| 44805/44805 [00:08<00:00, 5461.39it/s]
16/18 Inputting actual and predicted travel time and dwell time: 100%|██████████| 2267/2267 [00:13<00:00, 170.93it/s]
17/18 filling in missing data for actual_arrival_time: 100%|██████████| 2267/2267 [00:03<00:00, 672.05it/s]
11/18 Calculating the total number of nan values: 100%|██████████| 44805/44805 [00:37<00:00, 1186.71it/s]


Total new null values = 0.0


7/18 calculating average and total dwell times: 100%|██████████| 76/76 [00:04<00:00, 15.47it/s]
18/18 calculating percentage data missing for historical information: 100%|██████████| 44805/44805 [00:36<00:00, 1230.88it/s]
  0%|          | 3/48453 [00:00<27:35, 29.26it/s]d:\Wallace\wallace-azure-ml\DataPreprocessing\delay_mechanism_04.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset["delay_type_2_desc"][index] = delay_list
c:\Users\panuk\anaconda3\envs\wallace-delay\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
  0%|   

In [ ]:
data_next_1_station.to_csv("Data/data_next_1_station_"+from_loc+"_"+to_loc+"_"+data_year+".csv",index=False)